In [1]:
## Imports

import pandas as pd

import nltk

from sklearn.preprocessing import LabelEncoder

In [2]:
# Creating pandas dataframe from CSV
df = pd.read_csv("PPS_all.csv")

## Data Exploration

In [3]:
df.head()

,Year,Tier,Issue/Country,Impact,Likelihood,Description
0,2019,1,cyberattack,high,moderate,A highly disruptive cyberattack on U.S. critic...
1,2019,1,North Korea,high,moderate,Renewed tensions on the Korean Peninsula follo...
2,2019,1,Iran,high,moderate,An armed confrontation between Iran and the Un...
3,2019,1,South China Sea,high,moderate,An armed confrontation over disputed maritime ...
4,2019,1,terrorist attack,high,moderate,A mass casualty terrorist attack on the U.S. h...


In [4]:
df.shape

(267, 6)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 6 columns):
Year             267 non-null int64
Tier             267 non-null int64
Issue/Country    267 non-null object
Impact           210 non-null object
Likelihood       210 non-null object
Description      267 non-null object
dtypes: int64(2), object(4)
memory usage: 12.6+ KB


In [6]:
# Changing the Impact and Likelihood columns to be categories instead of strings
category_columns = ["Impact", "Likelihood"]

for cat_col in category_columns:
    df[cat_col] = df[cat_col].astype("category")
    # Changing the scale to be 1-3, instead of 0-2
    df[cat_col] = df[cat_col].cat.codes + 1

In [43]:
df.head()

,Year,Tier,Issue/Country,Impact,Likelihood,Description
0,2019,1,cyberattack,1,3,A highly disruptive cyberattack on U.S. critic...
1,2019,1,North Korea,1,3,Renewed tensions on the Korean Peninsula follo...
2,2019,1,Iran,1,3,An armed confrontation between Iran and the Un...
3,2019,1,South China Sea,1,3,An armed confrontation over disputed maritime ...
4,2019,1,terrorist attack,1,3,A mass casualty terrorist attack on the U.S. h...


In [51]:
years = list(range(2011, 2020))

annual_dict = {}

for obs in df.iterrows():
    for year in years:
        year_list = []
        if df["Year"][obs] == year:
            year_list.append(df["Description"][obs])
        annual_dict[year] = year_list

In [55]:
df.groupby(df["Year"]).sum()

,Tier,Impact,Likelihood
Year,,,
2011,55,0,0
2012,60,0,0
2013,60,67,79
2014,60,65,71
2015,60,64,74
2016,58,65,70
2017,63,67,78
2018,61,63,77
2019,63,66,77
